In [1]:
import openeo

In [2]:
openeo.client_version()

'0.13.0'

In [3]:
backend = "openeo.cloud"
conn = openeo.connect(backend).authenticate_oidc(provider_id='egi')

Authenticated using refresh token.


In [4]:
# Define a process graph
spatial_extent = {
    'west': 9.981079,
    'east': 12.958374,
    'south': 46.061797,
    'north': 47.428087,
    'crs': 'EPSG:4326',
}

temporal_extent = ["2021-01-01", "2021-01-02"]
collection = 'SENTINEL2_L1C'
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12']

cube = conn.load_collection(
    collection,
    spatial_extent=spatial_extent,
    bands=bands,
    temporal_extent=temporal_extent,
    fetch_metadata=False
)
ard = cube.ard_surface_reflectance(atmospheric_correction_method = 'FORCE', cloud_detection_method = 'Fmask', elevation_model = "cop-dem-30m")

data_saved = ard.save_result(format="NetCDF")

In [5]:
UC1 = data_saved.flat_graph()

In [8]:
job_ard = conn.create_job(UC1)
job_id_ard = job_ard.job_id

In [13]:
job_ard.start_job()

In [14]:
if job_id_ard:
    print("Batch job created with id: ", job_id_ard)
    job_ard.start_job()
else:
    print("Error! Job ID is None")

Batch job created with id:  eodc-jb-e698c53b-8e3d-476c-92ce-64320c8568a9


In [11]:
import time

In [ ]:
job_stat = True
while job_stat:
    
    try:
        if job_ard.status() == "running" or job_ard.status() == "queued":
            job_stat = True
        else:
            job_stat = False
    except:
        pass
            
            
    time.sleep(60)
    print(str(job_ard.status())+" -", end= " ")
# print(datetime.datetime.now())
# end = datetime.datetime.now()

running - running - running - running - running - running - running - running - 

Connection with expired access token ([403] TokenInvalid) automatically re-authenticated with refresh token.


running - running - running - running - running - running - running - running - running - running - running - 

In [39]:
results = job_ard.get_results()
metadata = results.get_metadata()
results.download_files("./")

In [56]:
job_ard.status()

'queued'

In [20]:
job_ard.stop_job()